# Validate NN on VMH unseen database
<b>Author</b>: Ian Coleman <br/>
<b>Function</b>: Let's take the NN developed in Opa/ and test it out on virtual metabolic human data

In [312]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from random import randint
import random
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from pandas_ml import ConfusionMatrix
import json
import subprocess
import pickle
import math
import scipy.io
import urllib.request
import pickle


#Set random seed
np.random.seed(1606)

### Import VMH Disease List

In [313]:
# Import VMH diseases list
v_dis = pd.read_csv('data/recon-store-diseases-1.tsv', sep='\t')

In [314]:
v_dis.shape

(257, 27)

In [315]:
v_dis.head()

,abbreviation,name,dtype,subtype,inheritance,organ,omim,prevalence,references,ghr,...,genereviews,clingendosage,igsr1000genoms,gwascataloge,gwascentral,geno2mp,clinvar,lovd,malacard,omim_symptons
0,BS1,"Bartter Syndrome, Type 1",Inherited metabolic disorder,Sodium-potassium-chloride cotransporter,Autosomal recessive,Kidney,601678,NaN,PMID: 17998760,bartter-syndrome,...,NaN,SLC12A1,ENSG00000074803,SLC12A1,SLC12A1,SLC12A1,600839.0,SLC12A1,bartter_syndrome_type_1_antenatal,601678.0
1,3MGA,3-Methylglutaconic Aciduria Type I,Inherited metabolic disorder,Organic acid disorders,Autosomal recessive,Nervous system,250950,Fewer than 20 cases,"The American Journal of Human Genetics, Volume...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,doi:10.1086/344712,3-methylglutaconyl-coa-hydratase-deficiency,67046,NaN,3-Methylglutaconic_aciduria,NaN,10906,7606,10321,NaN,...,3_methylglutaconic_aciduria_type_i_2,250950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FLIPO,Farber Lipogranulomatosis,Inherited metabolic disorder,Lysosomal disorder,Autosomal recessive,"Skin, skeletal system, liver, spleen, nervous ...",228000,Farber lipogranulomatosis is a very rare disor...,PMID: 19250404,farber-lipogranulomatosis,...,NaN,ASAH1,ENSG00000104763,ASAH1,ASAH1,ASAH1,613468.0,ASAH1,farber_lipogranulomatosis,228000.0
4,ACAD,Acetyl-Coa Carboxylase Deficiency,Inherited metabolic disorder,Lipid disorder,NaN,"Muscle, nervous system",200350,Only 1 case of ACC deficiency has been reporte...,PMID: 6114432,NaN,...,NaN,ACACA,ENSG00000275176,ACACA,ACACA,ACACA,200350.0,ACACA,acetyl_coa_carboxylase_deficiency,NaN


In [316]:
# v_dis = v_dis[['abbreviation', 'name', 'dtype', 'subtype', 'organ', 'omim', 'ghr', 'orphanet',
#  'cellLines',
#  'clinicaltrials',
#  'eurogenetest',
#  'geneticalliance',
#  'gard',
#  'igsr1000genoms',
#  'gwascentral',
#  'clinvar',
#  'malacard',
#  'omim_symptons']]

In [317]:
v_dis = v_dis[['abbreviation', 'name', 'omim', 'gwascentral']]

In [318]:
v_dis = v_dis.drop(v_dis.index[[2]])

In [319]:
v_dis['abbreviation'] = v_dis.abbreviation.astype(str)
v_dis['gwascentral'] = v_dis.gwascentral.astype(str)

### Map in mesh names

In [320]:
diseases = pd.read_csv('../ctd-to-nt/csvs/CTD_diseases.csv', skiprows=27)

In [321]:
# # Use a measure of distance to match up disease names from ctd (MESH) and from sider (UMLS) 
# from difflib import SequenceMatcher
# import pdb

# def similar(a, b):
#     return SequenceMatcher(None, a, b).ratio()

# def create_map(std_list, flawed_list):
#     flawed_list = (n for n in flawed_list)
#     team_map = {}
#     best_score = {}
#     for team in flawed_list:
#         scores = [similar(team, std_team) for std_team in std_list]
#         highest = max(scores)
#         if highest > 0.8:
#             index = scores.index(max(scores))
#             team_map[team] = std_list[index]
#     return team_map

# # Make map
# orig = sorted(v_dis.name.unique())
# comp = sorted(diseases['# DiseaseName'].unique())
# dis_mesh_map = create_map(orig, comp)

# # # These are the incorrect mappings I've identified for a 0.8 similarity cutoff
# remove = ('18-Hydroxylase deficiency', '3-Methylglutaconic Aciduria Type IV', '3-Methylglutaconic Aciduria, Type V',
#          'Bartter Syndrome, Type 4A', 'Bartter Syndrome, Type 4b', 'Bartter syndrome, type 3', 
#          'Biotin deficiency', 'Bonneau Syndrome', 'Brunoni syndrome', 'Carnitine Acetyltransferase Deficiency', 
#          'Carnitine Palmitoyltransferase II Deficiency, Infantile', 'Carnitine Palmitoyltransferase II Deficiency, Late-Onset', 
#          'Carnitine Palmitoyltransferase II Deficiency, Lethal Neonatal', 'Congenital Disorder Of Glycosylation, Type IIB',
#           'Congenital Disorder Of Glycosylation, Type IIF',  'Congenital Disorder Of Glycosylation, Type IIH', 
#           'Congenital Disorder Of Glycosylation, Type IIID',  'Congenital Disorder Of Glycosylation, Type Im',
#          'Darier Disease', 'Down Syndrome', 'Enterokinase Deficiency', 'Gardner Syndrome', 'Glycoprotein Storage Disease',
#          'Horner Syndrome', 'Lactate Dehydrogenase B Deficiency', 'Lactate Dehydrogenase Deficiency', 'Laron Syndrome',
#          'Leriche Syndrome', 'Lipoid congenital adrenal hyperplasia', 'Malonic aciduria', 'Meige Syndrome', 'Meigs Syndrome',
#          'Prolidase Deficiency', 'Rh Deficiency Syndrome', 'Silicosis', 'Sweet Syndrome', 'Triosephosphate Isomerase Deficiency', 
#          'Turner Syndrome', 'Werner Syndrome', 'Xylosidase Deficiency')
# dis_mesh_map = {key: dis_mesh_map[key] for key in dis_mesh_map if key not in remove}

# # Export map of UMLS:MESH
# with open('disname_mesh_map'+ '.pkl', 'wb') as f:
#         pickle.dump(dis_mesh_map, f, pickle.HIGHEST_PROTOCOL)

In [322]:
# Loading the map from pickle object - if you haven't created it you may need to uncomment above lines
def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

dis_mesh_map = load_obj('disname_mesh_map')

In [323]:
len(dis_mesh_map)

211

In [324]:
# Now map the MESH name to MESH ID
v_dis['MESH'] = v_dis.name.map(lambda x: dis_mesh_map.get(x))
name2meshID = dict(zip(diseases['# DiseaseName'], diseases.DiseaseID))
v_dis['MESH'] = v_dis.MESH.map(lambda x: name2meshID.get(x))

# Make am abbrev to mesh map for later
abbrev2mesh = dict(zip(v_dis.abbreviation, v_dis.MESH))

In [325]:
v_dis.head()

,abbreviation,name,omim,gwascentral,MESH
0,BS1,"Bartter Syndrome, Type 1",601678,SLC12A1,None
1,3MGA,3-Methylglutaconic Aciduria Type I,250950,nan,None
3,FLIPO,Farber Lipogranulomatosis,228000,ASAH1,MESH:D055577
4,ACAD,Acetyl-Coa Carboxylase Deficiency,200350,ACACA,MESH:C562678
5,PALD,Pseudoneonatal Adrenoleukodystrophy,264470,ACOX1,None


In [326]:
print(v_dis.shape) # 255
v_dis.MESH.nunique()

(256, 5)


49

### Get paired chems for these diseases
Using the above disease abbreviation, API call for associated chems from VMH

In [327]:
import pdb
def get_met(abbrev):
    """
    2nd attempt - this time create a line for every metabolite
    Given a disease abbreviation from VMH make api call to get assoc metabolite
    """
    url = 'https://www.vmh.life/_api/biomarkers/?disease=' + abbrev
    response = urllib.request.urlopen(url).read().strip() # make api call 
    response = json.loads(response) # turn from byte to dict
    metName, value, cid, chebi, chembl = [np.nan] * 5
    for metabolite in response['results']:
        try:
            value = metabolite['value']
            cid = metabolite['metabolite']['pubChemId']
            chebi = metabolite['metabolite']['cheBlId']
            chembl = metabolite['metabolite']['chembl']
            metName = metabolite['metabolite']['fullName']
            metGene = metabolite['iem']['gwascentral']
        except Exception as e:
            print('Exception: ', e)
            return [abbrev, metName, value, cid, chebi, chembl, metGene]
            pass
        if value == 'Increased':
            return [abbrev, metName, value, cid, chebi, chembl, metGene]
        else:
            return ([np.nan] * 6)

In [328]:
# vmh_df = pd.DataFrame()
# vmh_df['column'] = v_dis.abbreviation.apply(get_met)
# vmh_df = vmh_df[vmh_df.column.map(lambda x: x is not None)]
# vmh_df[['DiseaseAbbrev', 'metName', 'value', 'cid', 'chebi', 'chembl', 'MetGene']] = pd.DataFrame(vmh_df.column.values.tolist(), index= vmh_df.index)
# del vmh_df['column']
# del vmh_df['value']
# vmh_df.to_csv('vmh_df.csv', index=False)
# vmh_df.shape # 40, 6

In [329]:
vmh_df = pd.read_csv('vmh_df.csv')

In [330]:
vmh_df.head()

,DiseaseAbbrev,metName,cid,chebi,chembl,MetGene
0,3MGA,3-Hydroxy-Isovaleryl Carnitine,53915061.0,73027.0,NaN,AUH
1,ARG,L-Arginine,6322.0,16467.0,CHEMBL1485,ARG1
2,ASA,Citrulline,9750.0,16349.0,CHEMBL444814,ASL
3,BTD,Propionylcarnitine,107738.0,28867.0,NaN,BTD
4,BKT,Tiglyl Carnitine,22833596.0,NaN,NaN,ACAT1


In [331]:
vmh_df.shape # 40, 6

(40, 6)

In [332]:
vmh_df.reset_index(inplace=True, drop=True)
vmh_df['MetGene'] = vmh_df.MetGene.astype(str)
vmh_df['cid'] = vmh_df.cid.astype(str)

vmh_df.head()

,DiseaseAbbrev,metName,cid,chebi,chembl,MetGene
0,3MGA,3-Hydroxy-Isovaleryl Carnitine,53915061.0,73027.0,NaN,AUH
1,ARG,L-Arginine,6322.0,16467.0,CHEMBL1485,ARG1
2,ASA,Citrulline,9750.0,16349.0,CHEMBL444814,ASL
3,BTD,Propionylcarnitine,107738.0,28867.0,NaN,BTD
4,BKT,Tiglyl Carnitine,22833596.0,NaN,NaN,ACAT1


In [333]:
#  # Exploring api\n",	
# url = 'https://www.vmh.life/_api/biomarkers/?disease=TFPD'
# response = urllib.request.urlopen(url).read().strip()
# json.loads(response)['results']
#     # response['results'][0]

### Get gofuncs 
With the single gene given by VMH


In [334]:
# Export gene name list 
genes = list(v_dis.gwascentral) + list(vmh_df.MetGene)
np.savetxt(r'v_genes.txt', list(set(genes)), fmt='%s')
# set(genes)

#### NOTE the next step is MANUAL
You need to go to https://www.uniprot.org/uploadlists/ and give it the created v_genes.txt file, ask it to convert
Gene Names to uniprot ID. Then download this as UniprotIDs.tab (as uncompressed, mapping table) to this folder

In [335]:
# Import manually generated file of geneID --> uniprotID
df_uni_ids = pd.read_csv('data/UniprotIDs.tab', sep='\t',usecols=[0,1])
df_uni_ids.columns = ['GeneID', 'UniprotID']
df_uni_ids['GeneID'] = df_uni_ids.GeneID.astype(str)
df_uni_ids['UniprotID'] = df_uni_ids.UniprotID.astype(str)

In [336]:
df_uni_ids.sample(3)

,GeneID,UniprotID
1432,ASAH1,A0A1B0GU06
1010,GCDH,A0A2L2DH25
677,SUCLA2,A0A0U1RQL1


In [337]:
# Create mapping file of gene ID to chem ID... and gene ID to disease ID 
cg_dict = dict(zip(vmh_df.MetGene, vmh_df.cid))
cgd_dict = dict(zip(v_dis.gwascentral, v_dis.abbreviation))

In [338]:
df_uni_ids_d = df_uni_ids.copy() # make a copy for diseases

In [339]:
df_uni_ids.head()

,GeneID,UniprotID
0,HPD,A0A0B4J1R4
1,HPD,P32754
2,GALNS,A0A1A7UP97
3,GALNS,A0A286S6D8
4,GALNS,A0A2I6U1L6


In [340]:
# Apply the mapping, thereby creating chemical column
df_uni_ids['ChemicalID'] = df_uni_ids.GeneID.astype(str)
df_uni_ids['ChemicalID'] = df_uni_ids['ChemicalID'].astype(str).map(cg_dict)

In [341]:
# Apply the mapping, thereby creating disease column
df_uni_ids_d['DiseaseID'] = df_uni_ids_d.GeneID
df_uni_ids_d['DiseaseID'] = df_uni_ids_d['DiseaseID'].astype(str).map(cgd_dict)

In [342]:
df_uni_ids = df_uni_ids.dropna()
df_uni_ids_d = df_uni_ids_d.dropna()
print(df_uni_ids.shape) # 190, 3
print(df_uni_ids_d.shape) # 1870, 3

(223, 3)
(1870, 3)


In [343]:
df_uni_ids.sample(2)

,GeneID,UniprotID,ChemicalID
51,HADHA,A0A2R8Y688,168381.0
182,CPS1,Q5R209,5961.0


In [344]:
df_uni_ids_d.sample(14)

,GeneID,UniprotID,DiseaseID
1620,PAH,F8W0A0,PKU
1093,BCAT2,M0QXF9,HYPVLI
1783,DLD,E9PEX6,MSUD
930,ARSA,V9GYR0,METLEU
280,SCP2,H0YEU8,ZSLEU
281,SCP2,H0YF61,ZSLEU
1381,PRODH,X5D7N2,HYPRO1
1427,ASAH1,A0A1B0GTM3,FLIPO
220,MPI,H3BP57,CDG1B
61,ALDH4A1,Q5TF55,HPII


In [345]:
# # Output an association file for each of chem and dis
# df_c = df_uni_ids[['ChemicalID', ]]

# np.savetxt(r'VMHassociations_c.txt', df_c.values, fmt='%s')
# np.savetxt(r'VMHassociations_d.txt', df_d.values, fmt='%s')

In [346]:
# # Let's use this gene-uniprot df as our base, add disease into it via map
# gen2dis = dict(zip(v_dis.gwascentral, v_dis.omim))
# df_uni_ids['OMIM'] = df_uni_ids.GeneID.map(lambda x: gen2dis.get(x))

In [347]:
# Now turn unprot to go func
# import goa file (uniprot ID to go_functions)
go_funcs = pd.read_csv('../goa_human.gaf', header=None, skiprows=30, sep='\t')

/home/ian/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [348]:
# Cut out all cols except uniprot ids and go_funcs, rename these
go_funcs = go_funcs.rename(columns={ go_funcs.columns[1]: "UniprotID" })
go_funcs = go_funcs.rename(columns={ go_funcs.columns[4]: "gofunc" })
col_list = ['UniprotID', 'gofunc']
df_go = go_funcs[col_list]

In [349]:
df_go.head()

,UniprotID,gofunc
0,A0A024R161,GO:0003924
1,A0A024R161,GO:0007186
2,A0A024RBG1,GO:0003723
3,A0A024RBG1,GO:0005829
4,A0A024RBG1,GO:0008486


In [350]:
df_uni_ids.head()

,GeneID,UniprotID,ChemicalID
0,HPD,A0A0B4J1R4,6057.0
1,HPD,P32754,6057.0
37,ACADVL,A0A077H1I5,168381.0
38,ACADVL,B3KPA6,168381.0
39,ACADVL,G3V1M7,168381.0


In [351]:
df_go['UniprotID'] = df_go.UniprotID.astype(str)
df_uni_ids['UniprotID'] = df_uni_ids.UniprotID.astype(str)
df_uni_ids_d['UniprotID'] = df_uni_ids_d.UniprotID.astype(str)

/home/ian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [352]:
# Merge the go functions into our existing chem-uniprotID and dis-uniprotID dfs
df_uni_ids_d = df_uni_ids_d.merge(df_go, on='UniprotID', how='outer').dropna()
df_uni_ids = df_uni_ids.merge(df_go, on='UniprotID', how='outer').dropna()

In [353]:
df_uni_ids.head()

,GeneID,UniprotID,ChemicalID,gofunc
1,HPD,P32754,6057.0,GO:0003868
2,HPD,P32754,6057.0,GO:0003868
3,HPD,P32754,6057.0,GO:0005829
4,HPD,P32754,6057.0,GO:0006559
5,HPD,P32754,6057.0,GO:0006559


In [354]:
df_uni_ids_d.head()

,GeneID,UniprotID,DiseaseID,gofunc
1,HPD,P32754,TYR3,GO:0003868
2,HPD,P32754,TYR3,GO:0003868
3,HPD,P32754,TYR3,GO:0005829
4,HPD,P32754,TYR3,GO:0006559
5,HPD,P32754,TYR3,GO:0006559


In [355]:
# Create a col with the full go url
df_uni_ids['go_url'] = '<' + 'http://purl.obolibrary.org/obo/' + df_uni_ids.gofunc.str.replace(':', '_')  + '>'
df_uni_ids_d['go_url'] =  '<' + 'http://purl.obolibrary.org/obo/' + df_uni_ids_d.gofunc.str.replace(':', '_')  + '>'

In [356]:
# Grab just the columns we want to output (diseaseID and go_url/ chemicalID and go_url)
col_list_c = ['ChemicalID', 'go_url']
col_list_d = ['DiseaseID', 'go_url']
df_c = df_uni_ids[col_list_c]
df_d = df_uni_ids_d[col_list_d]
df_c = df_c.drop_duplicates()
df_d = df_d.drop_duplicates()

In [357]:
# df_uni_ids[df_uni_ids.OMIM == np.nan]

In [358]:
df_d.head()

,DiseaseID,go_url
1,TYR3,<http://purl.obolibrary.org/obo/GO_0003868>
3,TYR3,<http://purl.obolibrary.org/obo/GO_0005829>
4,TYR3,<http://purl.obolibrary.org/obo/GO_0006559>
6,TYR3,<http://purl.obolibrary.org/obo/GO_0006572>
7,TYR3,<http://purl.obolibrary.org/obo/GO_0046872>


In [359]:
df_c.head()

,ChemicalID,go_url
1,6057.0,<http://purl.obolibrary.org/obo/GO_0003868>
3,6057.0,<http://purl.obolibrary.org/obo/GO_0005829>
4,6057.0,<http://purl.obolibrary.org/obo/GO_0006559>
6,6057.0,<http://purl.obolibrary.org/obo/GO_0006572>
7,6057.0,<http://purl.obolibrary.org/obo/GO_0046872>


In [360]:
# gene2gouri = dict(zip(df_uni_ids.GeneID, df_uni_ids.go_url))
# vmh_df
# Unforunately I now have to create a df of:
# Entity Gene
# Which may be awkward

In [361]:
# Output an association file for each of chem and dis
np.savetxt(r'VMHassociations_c.txt', df_c.values, fmt='%s')
np.savetxt(r'VMHassociations_d.txt', df_d.values, fmt='%s')

In [362]:
# Merge these two into one single file
subprocess.call('cat VMHassociations_c.txt > vmhassociations', shell=True)
subprocess.call('cat VMHassociations_d.txt >> vmhassociations', shell=True)

0

In [363]:
# Create entities.lst to inform opa2vec which entities we want vectors for
entities = df_d.DiseaseID.unique().tolist() + df_c.ChemicalID.unique().tolist()
# print(sorted(entities))
# np.savetxt(r'VMHentities.lst', entities, fmt='%s')

In [364]:
# Actually we have to make these into vectors at the same time as the opa-nn vecs so
# 1. get chem-gofuncs
# 2. integrate both into vec creation along with opa-nn vecs

### Get chem-gofuncs

In [365]:
# Does VMH give us chem-gene?
# - with current data
# - with other api call
# ELSE are these chems in CTD
# ELSE are these chems in DisGeNet

### Import the created vecs

In [366]:
# Import Gofunc vec file
with open('../opa/go-gofuncs.lst', 'r') as file:
    text = file.read()
    
# Strip and split vector data into list of lists [chem, vec]
text = text.replace('\n', '')
text = text.split(']')
text = [item.strip().split(' [') for item in text]

# Turn it into a data frame
df = pd.DataFrame(text)
df.columns = ['ID', 'Vector']

# Clean
df = df.dropna()
df['Vector'] = df.Vector.map(lambda x: x.rstrip().lstrip().replace('    ', ' ').replace('   ', ' ').replace('  ', ' ').replace(' ', ','))

# Turn vector column into a list
df['Vector'] = df.Vector.map(lambda x: x.split(','))
ent2vec = dict(zip(df.ID, df.Vector))

In [367]:
print('ID vec pairs: ', df.shape[0])
print('No. unique entities: ', df.ID.nunique())

ID vec pairs:  3412
No. unique entities:  3412


In [368]:
df.sample(10)

,ID,Vector
2606,D005472,"[-2.96734683e-02, -7.76007175e-02, -4.85067695..."
439,MESH:D019294,"[0.02657738, 0.13508515, 0.02618615, -0.133977..."
991,MESH:C567036,"[0.02693321, 0.10902761, 0.05104734, -0.114022..."
746,MESH:D010014,"[0.0293992, 0.12939522, 0.05346038, -0.1136768..."
2608,C000944,"[-0.0416164, -0.1550461, -0.38542432, -0.23598..."
1685,MESH:C566393,"[0.02737711, 0.09487388, 0.02540512, -0.097729..."
2186,MESH:D024821,"[-0.01296799, 0.10717503, 0.0324862, -0.123063..."
3154,MPS4A,"[2.55190413e-02, 1.32885084e-01, 4.48934175e-0..."
2724,D005054,"[0.0205101, 0.07917807, -0.19042702, -0.117317..."
2173,OMIM:617268,"[0.01571366, 0.1089543, 0.02160806, -0.1118552..."


In [369]:
# # Our ent2vec uses ctd chem id, so create a column of this
# # Load the dictionary object output in phen2vec
# def load_obj(name):
#     with open(name + '.pkl', 'rb') as f:
#         return pickle.load(f)

# ctd_cid_map = load_obj('../opa/ctd_cid_map')
# # Switch it around
# ctd_cid_map = {y.decode("utf-8"):x for x,y in ctd_cid_map.items()}


In [370]:
vmh_df.head()

,DiseaseAbbrev,metName,cid,chebi,chembl,MetGene
0,3MGA,3-Hydroxy-Isovaleryl Carnitine,53915061.0,73027.0,NaN,AUH
1,ARG,L-Arginine,6322.0,16467.0,CHEMBL1485,ARG1
2,ASA,Citrulline,9750.0,16349.0,CHEMBL444814,ASL
3,BTD,Propionylcarnitine,107738.0,28867.0,NaN,BTD
4,BKT,Tiglyl Carnitine,22833596.0,NaN,NaN,ACAT1


In [371]:
vmh_df.shape

(40, 6)

In [372]:
# Adapt cid to suit map
vmh_df['cid'] = vmh_df.cid.fillna(0)
vmh_df['cid'] = vmh_df.cid.astype(float)
vmh_df['cid'] = vmh_df.cid.fillna(0)
vmh_df['cid'] = vmh_df.cid.astype(int)
vmh_df['cid'] = vmh_df.cid.astype(str)

In [373]:
vmh_df['DVec'] = vmh_df.DiseaseAbbrev.map(lambda x: ent2vec.get(x))
vmh_df['CVec'] = vmh_df.cid.map(lambda x: ent2vec.get(x))

In [376]:
vmh_df['MESH'] = vmh_df.DiseaseAbbrev.map(lambda x: abbrev2mesh.get(x))
vmh_df['MESHDVec'] = vmh_df.MESH.map(lambda x: ent2vec.get(x))

In [309]:
vmh_df = vmh_df[vmh_df.DVec.map(lambda x: x is not None)]
vmh_df = vmh_df[vmh_df.CVec.map(lambda x: x is not None)]

In [310]:
vmh_df.shape

(31, 8)

In [378]:
vmh_df.sample(8)

,DiseaseAbbrev,metName,cid,chebi,chembl,MetGene,DVec,CVec,MESH,MESHDVec
21,GACR,L-Proline,145742,17203.0,NaN,OAT,"[0.02022498, 0.12082769, -0.00685551, -0.12484...","[0.00220228, 0.12708458, 0.02948296, -0.144197...",None,None
6,CIT1,Citrulline,9750,16349.0,CHEMBL444814,ASS1,"[0.02968795, 0.12731399, 0.01281689, -0.146728...","[-9.59014669e-02, 1.23284012e-01, -4.32338156e...",None,None
34,HCYS,L-Methionine,6137,16643.0,NaN,CBS,"[5.46227908e-03, 1.23684846e-01, -8.23141262e-...","[0.02158029, 0.0831786, -0.05406189, -0.171028...",MESH:D006712,None
2,ASA,Citrulline,9750,16349.0,CHEMBL444814,ASL,"[6.77401483e-01, 5.45091271e-01, -4.91399854e-...","[-9.59014669e-02, 1.23284012e-01, -4.32338156e...",MESH:D056807,"[0.01632265, 0.12623872, 0.04004899, -0.137581..."
5,CACT,Tetradecenoylcarnitine,53477791,84634.0,NaN,SLC25A20,"[3.13112102e-02, 1.19863659e-01, 4.72368449e-0...","[1.86810433e-03, 1.36842966e-01, 3.54562402e-0...",None,None
8,CPT1,L-Palmitoylcarnitine,11953816,17490.0,NaN,CPT1A,"[0.0077546, 0.12436599, 0.01024128, -0.1492381...","[-1.37756923e-02, 1.09985389e-01, 2.36444753e-...",None,None
1,ARG,L-Arginine,6322,16467.0,CHEMBL1485,ARG1,"[3.67329210e-01, -2.10704312e-01, -1.82017654e...","[-1.66938882e-02, 1.36134878e-01, -2.16508433e...",None,None
18,NaN,NaN,0,NaN,NaN,nan,None,None,None,None


### Add control rows

In [154]:
# Add correlation
vmh_df['Correlation'] = 1

In [155]:
# Add unrelated pairs - control obs
no_rows = (vmh_df.shape[0] +4)    # This is a parameter to be tuned --> how many uncorrelated pairs do we want
print('Original shape: ', vmh_df.shape)
vmh_df = vmh_df.drop_duplicates(subset=['DiseaseAbbrev', 'cid'], keep=False)
print('Shape after dropping duplicates: ', vmh_df.shape)

# Randomly select chems and diseases (as many as there are related pairs)
df_chems = vmh_df[['cid', 'CVec']].drop_duplicates(subset=['cid']).reset_index(drop=True)
df_dis = vmh_df[['DiseaseAbbrev', 'DVec']].drop_duplicates(subset=['DiseaseAbbrev']).reset_index(drop=True)
df_chems.columns = ['ID', 'Vector']
df_dis.columns = ['ID', 'Vector']

# print('chem size: ', df_chems.shape[0])
# print('dis size: ', df_dis.shape[0])

no_chems = len(df_chems) - 1
no_dis = len(df_dis) - 1
rand_chems = np.random.choice(no_chems, no_rows, replace=True)
rand_dis = np.random.choice(no_dis, no_rows, replace=True)

# Add the new pairs as rows
for x in range(0, no_rows):
    int1 = rand_chems[x]
    int2 = rand_dis[x]
    chem, chemvec = df_chems.loc[int1, 'ID'], df_chems.loc[int1, 'Vector']
    dis, disvec = df_dis.loc[int2, 'ID'], df_dis.loc[int2, 'Vector']
    vmh_df = vmh_df.append({'cid':chem, 'DiseaseAbbrev':dis, 'CVec':chemvec, 'DVec': disvec, 'Correlation':0}, ignore_index=True)

print('Shape after adding controls: ', vmh_df.shape)
# Drop any duplicates (removes known correlated pairs accidentally generated as uncorrelated)
vmh_df = vmh_df.drop_duplicates(subset=['cid', 'DiseaseAbbrev'], keep=False)
print('Shape after dropping duplicates: ', vmh_df.shape)

Original shape:  (31, 9)
Shape after dropping duplicates:  (31, 9)
Shape after adding controls:  (66, 9)
Shape after dropping duplicates:  (60, 9)


In [156]:
vmh_df.DiseaseAbbrev.unique()

array(['ARG', 'ASA', 'BTD', 'BKT', 'CACT', 'CIT1', 'CPS1', 'CPT1', 'CPT2',
       'HCS', 'HMG', 'TYR3', 'IBG', 'IVA', 'NICCD', 'GACR', 'PC', 'PKU',
       'SCAD', 'TYR2', 'VLCAD', 'TYR1', 'PA', 'NKH', 'MMA', 'HCYS',
       'HMET', 'GA2', 'MSUD', 'SBCADD', 'DECR'], dtype=object)

### Add HINO

In [157]:
vmh_df.sample(2)

,DiseaseAbbrev,metName,cid,chebi,chembl,MetGene,DVec,CVec,Correlation
19,TYR2,L-Tyrosine,6057,17895.0,CHEMBL925,TAT,"[0.01897371, 0.13630432, 0.03458931, -0.136873...","[-4.75196308e-03, 1.35927454e-01, 2.56152116e-...",1
41,CPT1,NaN,5961,NaN,NaN,NaN,"[0.0077546, 0.12436599, 0.01024128, -0.1492381...","[-0.04229089, 0.1120939, -0.00048257, -0.16405...",0


In [158]:
df_uni_ids.head()

,GeneID,UniprotID,ChemicalID,gofunc,go_url
1,HPD,P32754,6057.0,GO:0003868,<http://purl.obolibrary.org/obo/GO_0003868>
2,HPD,P32754,6057.0,GO:0003868,<http://purl.obolibrary.org/obo/GO_0003868>
3,HPD,P32754,6057.0,GO:0005829,<http://purl.obolibrary.org/obo/GO_0005829>
4,HPD,P32754,6057.0,GO:0006559,<http://purl.obolibrary.org/obo/GO_0006559>
5,HPD,P32754,6057.0,GO:0006559,<http://purl.obolibrary.org/obo/GO_0006559>


In [159]:
# Loading the map from pickle object - if you haven't created it you may need to uncomment above lines
def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

uniprot2hino = load_obj('../opa/Uniprot_HINO_map')

In [160]:
# The map has different format of uniprot ids, so adjust for that
df_uni_ids['UniprotID'] = df_uni_ids.UniprotID.map(lambda x: 'UniProt_' + x)
df_uni_ids_d['UniprotID'] = df_uni_ids_d.UniprotID.map(lambda x: 'UniProt_' + x)

In [161]:
df_uni_ids['HINO'] = df_uni_ids.UniprotID.map(lambda x: uniprot2hino.get(x))
df_uni_ids_d['HINO'] = df_uni_ids_d.UniprotID.map(lambda x: uniprot2hino.get(x))

In [162]:
df_uni_ids_d.sample(2)

,GeneID,UniprotID,DiseaseID,gofunc,go_url,HINO
7359,SLC17A5,UniProt_Q9NRA2,SIALAS,GO:0005765,<http://purl.obolibrary.org/obo/GO_0005765>,None
8241,G6PD,UniProt_P11413,G6PD,GO:0019322,<http://purl.obolibrary.org/obo/GO_0019322>,http://purl.obolibrary.org/obo/HINO_0024252


In [163]:
df_uni_ids.sample(2)

,GeneID,UniprotID,ChemicalID,gofunc,go_url,HINO
253,CPS1,UniProt_P31327,5961.0,GO:0019240,<http://purl.obolibrary.org/obo/GO_0019240>,None
731,FAH,UniProt_P16930,6057.0,GO:0006527,<http://purl.obolibrary.org/obo/GO_0006527>,http://purl.obolibrary.org/obo/HINO_0025022


In [164]:
# dis2hino = dict(zip(df_uni_ids_d.DiseaseID, df_uni_ids_d.HINO))
# chem2hino = dict(zip(df_uni_ids.ChemicalID, df_uni_ids.HINO))

In [165]:
# vmh_df['dis_HINOvec'] = vmh_df.DiseaseAbbrev.map(lambda x: dis2hino.get(x))
# vmh_df['chem_HINOvec'] = vmh_df.cid.map(lambda x: chem2hino.get(x))

In [166]:
# Create associations files
# Output an association file for each of chem and dis
df_uni_ids_d = df_uni_ids_d[df_uni_ids_d.HINO.map(lambda x: x is not None)]
df_uni_ids = df_uni_ids[df_uni_ids.HINO.map(lambda x: x is not None)]

np.savetxt(r'VMHassociations_disHINO.txt', df_uni_ids_d[['DiseaseID', 'HINO']].values, fmt='%s')
np.savetxt(r'VMHassociations_chemHINO.txt', df_uni_ids[['ChemicalID', 'HINO']].values, fmt='%s')

# Merge the two
subprocess.call('cat VMHassociations_disHINO.txt > VMHhinoAssociations', shell=True)
subprocess.call('cat VMHassociations_chemHINO.txt >> VMHhinoAssociations', shell=True)

0

In [167]:
# Make entities file
entities = df_uni_ids_d.DiseaseID.unique().tolist() + df_uni_ids.ChemicalID.unique().tolist()
np.savetxt(r'VMHHINOentities.lst', entities, fmt='%s')

#### MANUALLY run the hinoVecs notebook which will train these along with opa-nn HINO vecs

In [168]:
# Import HINO vec file
with open('../opa/hinoVecs.lst', 'r') as file:
    text = file.read()
    
# Strip and split vector data into list of lists [chem, vec]
text = text.replace('\n', '')
text = text.split(']')
text = [item.strip().split(' [') for item in text]

# Turn it into a data frame
df = pd.DataFrame(text)
df.columns = ['ID', 'Vector']

# Clean
df = df.dropna()
df['Vector'] = df.Vector.map(lambda x: x.rstrip().lstrip().replace('    ', ' ').replace('   ', ' ').replace('  ', ' ').replace(' ', ','))

# Turn vector column into a list
df['Vector'] = df.Vector.map(lambda x: x.split(','))

# Make a map of it (DisID to DOvec)
entity_to_HINOvec = dict(zip(df.ID, df.Vector))

In [169]:
vmh_df['dis_HINOvec'] = vmh_df.DiseaseAbbrev.map(lambda x: entity_to_HINOvec.get(x))
vmh_df['chem_HINOvec'] = vmh_df.cid.map(lambda x: entity_to_HINOvec.get(x))

### ADD PRO ont vecs

In [170]:
# Import PRO vec file
with open('../opa/PROVecs.lst', 'r') as file:
    text = file.read()
    
# Strip and split vector data into list of lists [chem, vec]
text = text.replace('\n', '')
text = text.split(']')
text = [item.strip().split(' [') for item in text]

# Turn it into a data frame
df = pd.DataFrame(text)
df.columns = ['ID', 'Vector']

# Clean
df = df.dropna()
df['Vector'] = df.Vector.map(lambda x: x.rstrip().lstrip().replace('    ', ' ').replace('   ', ' ').replace('  ', ' ').replace(' ', ','))

# Turn vector column into a list
df['Vector'] = df.Vector.map(lambda x: x.split(','))

# Make a map of it (DisID to DOvec)
entity_to_PROvec = dict(zip(df.ID, df.Vector))

In [171]:
vmh_df['dis_PROvec'] = vmh_df.DiseaseAbbrev.map(lambda x: entity_to_PROvec.get(x))
vmh_df['chem_PROvec'] = vmh_df.cid.map(lambda x: entity_to_PROvec.get(x))

### Add Phenotype Ontology Vecs

In [172]:
# diseases = pd.read_csv('../ctd-to-nt/csvs/CTD_diseases.csv', skiprows=27)

In [184]:
# diseases.sample(3)

,# DiseaseName,DiseaseID,AltDiseaseIDs,Definition,ParentIDs,TreeNumbers,ParentTreeNumbers,Synonyms,SlimMappings
9026,Occupational Diseases,MESH:D009784,NaN,Diseases caused by factors involved in one's e...,MESH:C,C24,C,"Disease, Occupational|Diseases, Occupational|I...",Occupational disease
4778,Flaviviridae Infections,MESH:D018178,NaN,Infections with viruses of the family FLAVIVIR...,MESH:D012327,C02.782.350,C02.782,"Flaviviridae Infection|Infection, Flaviviridae...",Viral disease
5429,Heavy Chain Disease,MESH:D006362,DO:DOID:0060125,A disorder of immunoglobulin synthesis in whic...,MESH:D008232|MESH:D010265,C15.378.147.780.490|C15.604.515.435|C20.683.78...,C15.378.147.780|C15.604.515|C20.683.780,Franklin Disease|Franklin's Disease|Franklins ...,Blood disease|Immune system disease|Lymphatic ...


In [187]:
# v_dis.sample(3)

,abbreviation,name,omim,gwascentral
248,LALD,Lysosomal Acid Lipase Deficiency,278000,LIPA
226,MSUD,Maple Syrup Urine Disease,248600,DLD
120,SIALAS,Sialic Acid Storage Disease,269920 and 604369,SLC17A5


### PreProcess

In [185]:
vmh_df.sample(12)

,DiseaseAbbrev,metName,cid,chebi,chembl,MetGene,DVec,CVec,Correlation,dis_HINOvec,chem_HINOvec,dis_PROvec,chem_PROvec
52,PA,NaN,6322,NaN,NaN,NaN,"[0.997726262, -0.211966112, 0.329555333, -2.40...","[-0.0166938882, 0.136134878, -0.0216508433, -0...",0,"[1.1072729, -0.27654296, 0.25561085, -2.413737...","[0.00634056935, 0.0369067043, 0.0183985457, -0...","[1.0172819, -0.41897404, 0.2862702, -2.1424572...","[0.00559254549, 0.0277997069, 0.0273977593, -0..."
28,MSUD,L-Valine,6287,16414.0,NaN,DLD,"[0.03215135, 0.14425296, 0.0082859, -0.1536295...","[0.0447932296, 0.140804142, 0.0266556181, -0.1...",1,"[0.01061395, 0.03882901, 0.00975791, -0.037047...","[0.0103904633, 0.0445794053, 0.0196699928, -0....","[0.0119402353, 0.0360205024, 0.0205472168, -0....","[0.00642648, 0.04142284, 0.01987912, -0.038156..."
32,PA,NaN,439829,NaN,NaN,NaN,"[0.997726262, -0.211966112, 0.329555333, -2.40...","[-0.01492804, 0.11784542, -0.0127729, -0.15774...",0,"[1.1072729, -0.27654296, 0.25561085, -2.413737...","[0.02103061, 0.06809001, 0.03241677, -0.061152...","[1.0172819, -0.41897404, 0.2862702, -2.1424572...","[0.0143094482, 0.0745006204, 0.0242867768, -0...."
6,CPS1,L-Glutamine,5961,18050.0,NaN,CPS1,"[-0.00828421488, 0.163499296, 0.00525511336, -...","[-0.04229089, 0.1120939, -0.00048257, -0.16405...",1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
62,BKT,NaN,6137,NaN,NaN,NaN,"[-0.06295022, 0.11790282, -0.03769448, -0.1673...","[0.02158029, 0.0831786, -0.05406189, -0.171028...",0,"[0.00961637, 0.05080085, 0.02324037, -0.040904...","[0.01819745, 0.05775932, 0.02726264, -0.060259...","[0.01152219, 0.03913221, 0.02425494, -0.045936...","[0.01765791, 0.04911795, 0.02775674, -0.048549..."
43,HMG,NaN,107738,NaN,NaN,NaN,"[-0.0167686325, 0.241707161, -0.155845612, 0.0...","[-0.00510412, 0.1282246, 0.02975616, -0.168319...",0,"[0.0624221712, 0.0975745022, -0.131599113, 0.0...","[0.0161357801, 0.038298618, 0.016508732, -0.03...","[0.13170993, 0.45835462, -0.33259922, 0.491003...","[0.0131762, 0.0415750369, 0.0240649115, -0.036..."
25,HCYS,L-Methionine,6137,16643.0,NaN,CBS,"[0.00546227908, 0.123684846, -0.00823141262, -...","[0.02158029, 0.0831786, -0.05406189, -0.171028...",1,"[0.0214955, 0.0428162, 0.02418486, -0.03971911...","[0.01819745, 0.05775932, 0.02726264, -0.060259...","[0.01634672, 0.02955634, 0.01747652, -0.031329...","[0.01765791, 0.04911795, 0.02775674, -0.048549..."
18,SCAD,Butyrylcarnitine,439829,NaN,NaN,ACADS,"[0.11330748, 0.16369171, -0.02563963, -0.13694...","[-0.01492804, 0.11784542, -0.0127729, -0.15774...",1,"[0.10702512, 0.11032967, -0.0422197, -0.068437...","[0.02103061, 0.06809001, 0.03241677, -0.061152...","[0.14656241, 0.08825016, -0.04374115, -0.10401...","[0.0143094482, 0.0745006204, 0.0242867768, -0...."
38,CACT,NaN,6140,NaN,NaN,NaN,"[0.0313112102, 0.119863659, 0.0472368449, -0.1...","[0.01009831, 0.13669972, 0.04413829, -0.135294...",0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.00879558921, 0.0505895354, 0.0275596697, -0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.01773384, 0.03808071, 0.0189811, -0.0452998..."
24,MMA,Propionylcarnitine,107738,28867.0,NaN,MMAA,"[0.0406323224, 0.743928432, -0.169774011, -0.4...","[-0.00510412, 0.1282246, 0.02975616, -0.168319...",1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0161357801, 0.038298618, 0.016508732, -0.03...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0131762, 0.0415750369, 0.0240649115, -0.036..."


In [175]:
# Need to turn all to float and add empty vecs
all_vecs = ['CVec', 'DVec', 'dis_HINOvec', 'chem_HINOvec', 'dis_PROvec', 'chem_PROvec']
empty_vec = [0] * 200
for col in all_vecs:
    vmh_df[col] = vmh_df[col].map(lambda x: empty_vec if x is None else x)
    vmh_df[col] = vmh_df[col].map(lambda x: [float(i) for i in x])

In [176]:
# For Keras, need to turn inputs into numpy arrays instead of pandas df
# First create single np array of all vecs... not pretty:
Dvecs = pd.DataFrame(vmh_df.DVec.values.tolist(), index= vmh_df.index)
Cvecs = pd.DataFrame(vmh_df.CVec.values.tolist(), index= vmh_df.index)
all_X = Dvecs.merge(Cvecs, how='outer', left_index=True, right_index=True)

# dHINOvecs = pd.DataFrame(vmh_df.dis_HINOvec.values.tolist(), index=vmh_df.index)
# cHINOvecs = pd.DataFrame(vmh_df.chem_HINOvec.values.tolist(), index=vmh_df.index)
# hinovecs = cHINOvecs.merge(dHINOvecs, how='outer', left_index=True, right_index=True)
# all_X = all_X.merge(hinovecs, how='outer', left_index=True, right_index=True)

dPROvecs = pd.DataFrame(vmh_df.dis_PROvec.values.tolist(), index=vmh_df.index)
cPROvecs = pd.DataFrame(vmh_df.chem_PROvec.values.tolist(), index=vmh_df.index)
PROvecs = cPROvecs.merge(dPROvecs, how='outer', left_index=True, right_index=True)
all_X = PROvecs.merge(all_X, how='outer', left_index=True, right_index=True)


all_X = np.array(all_X)

In [177]:
# Now create np array of the y output
all_y = np.array(vmh_df.Correlation)

In [178]:
print('y shape: ', all_y.shape)
print('X shape: ', all_X.shape)

y shape:  (60,)
X shape:  (60, 800)


### Import Model and Run Opa

In [179]:
# Load model (saved in opa-nn notebook)
from tensorflow.keras.models import load_model
# nn14022019auc921GoPhenCHEdoHI
# nn15022019auc937GoPhenCHEdoHIpro
# nn14022019auc921GoPhenCHEdoHI
# nn15022019auc92GoPhenCHEdoHIpro
# nn18022019auc87Go --> 0.619
# nn18022019auc87Go --. 0.608
# nn18022019auc876Go --> 0.615
model = load_model('../opa/nn20022019auc889GOpro.h5')

In [180]:
# 2. Compile the model (give it loss func, optimise func and eval metric)
model.compile(optimizer=tf.train.AdamOptimizer(), # determines how the model is adapted based on loss func
              loss='binary_crossentropy', # measure of accuracy during training
              metrics=['accuracy']) # measure for train and testing steps 

In [181]:
# Accuracy
test_loss, test_acc = model.evaluate(all_X, all_y)
print('Test accuracy:', test_acc)

60/60 [==============================] - 0s 2ms/step
Test accuracy: 0.4666666626930237


In [182]:
# Get actual predictions for test set
predictions = model.predict(all_X)
rounded_predictions = [int(float(round(x[0]))) for x in predictions]

In [183]:
# ROC AUC
print('ROC AUC: ', roc_auc_score(all_y, predictions))

ROC AUC:  0.5984427141268075


### Get assoc genes for diseases (ii) pair them to CTD via semantic matching/omim

In [580]:
# # Map the vmh disease abbreviations to our disease list
# # import diseases
# c_dis = pd.read_csv('../ctd-to-nt/all-diseases-w-genes-ctd-idsnamesgenes.csv')
# print('CTD chems: ', c_dis.shape[0])

In [581]:
# c_dis.head()

In [582]:
# # Match up diseases in two ways, OMIM and semantic similarity (may be ~ no matches...)
# # Use a measure of distance to match up disease names from ctd and from VMH 
# from difflib import SequenceMatcher
# import pdb

# def similar(a, b):
#     return SequenceMatcher(None, a, b).ratio()

# def create_map(std_list, flawed_list):
#     flawed_list = (n for n in flawed_list)
#     team_map = {}
#     best_score = {}
#     for team in flawed_list:
#         scores = [similar(team, std_team) for std_team in std_list]
#         highest = max(scores)
#         if highest > 0.8:
#             index = scores.index(max(scores))
#             team_map[team] = std_list[index]
#     return team_map

In [583]:
## KEEP commented out unless you haven't made this map yet
# mapboy = create_map(v_dis.name, c_dis.DiseaseName.unique())

# # Here we export the dictionary in a way that's easily imported as dict
# import pickle 

# with open('Uniprot_HINO_map'+ '.pkl', 'wb') as f:
#         pickle.dump(mapboy, f, pickle.HIGHEST_PROTOCOL)

In [584]:
# ## The commented section above makes a map of vmh dis to ctd dis, importing here (computationally expensive) 
# def load_obj(name):
#     with open(name + '.pkl', 'rb') as f:
#         return pickle.load(f)

# vdis2cdis = load_obj('vmhdis_to_ctddis')

In [585]:
# vdis2cdis

In [586]:
# # These are the incorrect mappings I've identified for a 0.8 similarity cutoff
# remove = ('Turner Syndrome', 'Werner Syndrome', 'Enterokinase Deficiency', 'Prolidase Deficiency')
# vdis2cdis = {key: vdis2cdis[key] for key in vdis2cdis if key not in remove}

In [587]:
# sorted(c_dis.DiseaseName.unique())[700:]

In [588]:
# sorted(v_dis.name)

In [589]:
# API script to get disease-marker pairs from VMH